In [ ]:
!pip install langchain_google_vertexai langgraph langchain_google_community

# Natural Language to SQL

In order to demonstrate a Natural Language to SQL architecture, lets first create a sample database.

We wull be using sqlite with the `langchain_google_community` `Database` interface.

Let's create an empty database in the local file `database.sqlite`:

In [2]:
from langchain_google_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///database.sqlite")

result = db.run(
    """
      CREATE TABLE ITEMS
      (name TEXT, type TEXT, color TEXT, season TEXT, price FLOAT, description TEXT)
    """
)

Bear in mind that if the file already exists, you can get an error.

In order to populate the table, let's now read the data from `data.json`. This file contains item descriptions with some metadta, such as the type, color, season and prince of each item. 

Once the data is read, we insert each record in the database we just created.

In [3]:
import json

with open("data.json") as input_file:
    items = json.load(input_file)

sanitize = lambda string: string.replace("'", "") # Remove apostrophe
records = [
    ", ".join(
        f"'{sanitize(value)}'" if isinstance(value, str) else str(value)
        for colname, value in item.items()
    ) 
    for item in items
]

for record in records:
    statement = f"INSERT INTO ITEMS VALUES ({record})"
    try:
        db.run(statement)
    except Exception as err:
        print(statement)

Now let's define the tool the agent is going to use. The docstring of the function is the description that the model is going to use as the prompt, so is important for it to be descriptive. We include the schema of the database to make sure the agent knows which columns and datatypes exist.

In [4]:
from langchain_core.tools import tool

@tool
def execute_query(query: str) -> str:
    """ Executes a query against the clothing items database.
    The schema of the database is the following:
    ```
        CREATE TABLE ITEMS
        (
            name TEXT, -- Name of the item
            type TEXT, -- Type of the item can be 't-shirt', 'pants' or 'coat'
            color TEXT, -- Color of the item. Can be "red", "blue", "black" or "white"
            season TEXT, -- Can be "summer", "winter" "fall" or "spring"
            price FLOAT, -- Prize in dollars.
            description TEXT -- Description of the item
        )
    """
    return db.run_no_throw(query)

Now, we create the agent architecture, just as described in the book Chapter, and compile the graph to make it invokable.

In [6]:
from typing import Literal
from langchain_google_vertexai import ChatVertexAI
from langchain_core.messages import AnyMessage
from langgraph.graph import StateGraph, MessagesState, END
from langgraph.prebuilt import ToolNode
from langchain_core.messages import HumanMessage, SystemMessage


graph = StateGraph(MessagesState)

tools = [execute_query]

generator = ChatVertexAI(
  model_name="gemini-1.5-pro-001", 
  temperature = 0
).bind(tools=tools)

def invoke_generator(state: MessagesState) -> None:
    """ Represents the generator node.
    """
    response = generator.invoke(state["messages"])
    state["messages"].append(response)
    
def use_execute_query(state: MessagesState) -> Literal["query_tool", END]:
    """ Represents the use_retrieval conditional edge
    """
    if not state["messages"]:
        return END
    if state["messages"][-1].tool_calls:
        return "query_tool"
    return END


graph.add_node("generator", invoke_generator)
graph.add_node("query_tool", ToolNode(tools))
graph.set_entry_point("generator")

graph.add_conditional_edges("generator", use_execute_query)
graph.add_edge("query_tool", "generator")


agentic_sql = graph.compile()


We can query the agent with the `stream` method and obserb how it generates the query and executes it in the dabase, retrieving the results.

In [7]:

messages = [
    SystemMessage(
        """
        - You are a useful assistant that help users navigate a catalog of clothing items.
        - You can retrieve clothing items from the catalog using a SQL query.
        - Answer in natural language and format your output using paragraph or bullet points if
          necessary.
        """
    ),
    HumanMessage("Can you show me the name and price of all coats?")
]

stream_generator = agentic_sql.stream({"messages": messages}, stream_mode="values")

for state in stream_generator:
    state["messages"][-1].pretty_print()
state["messages"][-1].pretty_print()

================================ Human Message =================================

Can you show me the name and price of all coats?
================================= Tool Message =================================
Name: execute_query

[('Midnight Magic Coat', 89.99), ('Autumn Hues Sweater', 49.99), ('Winter Wonderland Jacket', 79.99), ('Cozy Cabin Cardigan', 59.99), ('Fall Foliage Vest', 49.99), ('Snowy Peaks Parka', 99.99)]
================================== Ai Message ==================================

Here are the coats with their prices:

* Midnight Magic Coat - $89.99
* Autumn Hues Sweater - $49.99
* Winter Wonderland Jacket - $79.99
* Cozy Cabin Cardigan - $59.99
* Fall Foliage Vest - $49.99
* Snowy Peaks Parka - $99.99


It can also execute calculations leveraging `SQL` capabilities.

In [8]:

messages = [
    SystemMessage(
        """
        - You are a useful assistant that help users navigate a catalog of clothing items.
        - You can retrieve clothing items from the catalog using a SQL query.
        - Answer in natural language and format your output using paragraph or bullet points if
          necessary.
        """
    ),
    HumanMessage("What is the name and price in euros of the most expensive coat, given that 1 euro is currently 1.1 dollars?")
]

stream_generator = agentic_sql.stream({"messages": messages}, stream_mode="values")

for state in stream_generator:
    state["messages"][-1].pretty_print()
state["messages"][-1].pretty_print()

================================ Human Message =================================

What is the name and price in euros of the most expensive coat, given that 1 euro is currently 1.1 dollars?
================================= Tool Message =================================
Name: execute_query

[('Snowy Peaks Parka', 109.989)]
================================== Ai Message ==================================

The most expensive coat is the Snowy Peaks Parka and its price is 109.99 euros.
